### Rail Fence Cipher

&nbsp;

Rail fence cipher is one of the easiest transposition ciphers. Basically we just write the plaintext following zigzag shape in the matrix and fill other elements with random characters. The cipher text can be extracted via flattening the matrix. 

For instance, the word `AIR` in a 2 by 3 matrix.

$$
\begin{pmatrix} A & ? & R\\ ? & I & ? \end{pmatrix}
$$

Once we fill the empty elements, it becomes

$$
\begin{pmatrix} A & T & R\\ A & I & L \end{pmatrix}
$$

Flatten the matrix to extract the ciphertext `ATRAIL`. Dividing ciphertext into two blocks, which is `ATR AIL`, will make things more confusing. Without knowing the number of rows, cryptanalysis may be a bit difficult. The easiest way is to brute force all the possible values by factorization of the length of the ciphertext.

In [1]:
import itertools
import numpy as np
import re

### Functions

In [2]:
#to make the ciphertext less revealing
#we can use lower case,remove space and punctuations via regex
#then group the letters by certain number
def break_into_blocks(ciphertext,bandwidth=6):

    #only capture words
    tough_ciphertext=list(map(lambda x:x.lower(),re.findall('\S',ciphertext)))
    
    #break into blocks
    new_ciphertext=[''.join(tough_ciphertext[i:i+bandwidth]) for i in range(0,len(tough_ciphertext),bandwidth)]

    #fill up the last block with padding
    if len(new_ciphertext[-1])<bandwidth:
        new_ciphertext[-1]+='a'*(bandwidth-len(new_ciphertext[-1]))
        
    #create output
    ultimate_ciphertext=' '.join(new_ciphertext)
    
    return ultimate_ciphertext

In [3]:
#create a random matrix with ascii number
#convert to characters
#create a pointer
#fill elements with the plaintext under the guidance of the pointer#
def rail_fence_cipher_encrypt(plaintext,num_of_rail=3):

    #initialize
    matrix=np.random.randint(low=33,high=127,size=(num_of_rail,len(plaintext)))
    vec=np.vectorize(lambda x:chr(x))
    matrix=np.apply_along_axis(vec,axis=1,arr=matrix)
    row_ind,col_ind=0,0
    row_dir='down'

    #the matrix is filled under the guidance of the pointer
    while col_ind<len(plaintext):

        #fill elements downwards diagonally
        if row_dir=='down':
            if row_ind<num_of_rail:
                matrix[row_ind][col_ind]=plaintext[col_ind]
                
            #if the floor is reached
            #reverse the pointer direction
            else:
                row_ind-=2
                row_dir='up'
                matrix[row_ind][col_ind]=plaintext[col_ind]
                
        #fill elements upwards diagonally
        elif row_dir=='up':
            if row_ind>=0:
                matrix[row_ind][col_ind]=plaintext[col_ind]
                
            #if the ceiling is reached
            #reverse the pointer direction
            else:
                row_ind+=2
                row_dir='down'
                matrix[row_ind][col_ind]=plaintext[col_ind]
        else:
            pass    

        #iterate through elements
        if row_dir=='down':
            row_ind+=1
        else:
            row_ind-=1
        col_ind+=1
    
    #create output
    ciphertext=''.join(matrix.ravel())
    
    return ciphertext

In [4]:
#the decryption is pretty much the same as encryption
#just grab elements diagonally following the pointer
def rail_fence_cipher_decrypt(ciphertext,num_of_rail=3):
    
    #initialize
    row_ind,col_ind=0,0
    row_dir='down'
    plaintext_list=[]

    #convert blocks into matrix
    matrix=np.array([list(ciphertext[i:i+int(len(ciphertext)/num_of_rail)]) for i in range(0,len(ciphertext),int(len(ciphertext)/num_of_rail))]).reshape(num_of_rail,int(len(ciphertext)/num_of_rail))

    #the matrix traversal is under the guidance of the pointer
    while col_ind<matrix.shape[1]:

        #move downwards diagonally
        if row_dir=='down':
            if row_ind<num_of_rail:
                plaintext_list.append(matrix[row_ind][col_ind])
                
            #if the floor is reached
            #reverse the pointer direction
            else:
                row_ind-=2
                row_dir='up'
                plaintext_list.append(matrix[row_ind][col_ind])
                
        #move upwards diagonally
        elif row_dir=='up':
            if row_ind>=0:
                plaintext_list.append(matrix[row_ind][col_ind])
                
            #if the ceiling is reached
            #reverse the pointer direction
            else:
                row_ind+=2
                row_dir='down'
                plaintext_list.append(matrix[row_ind][col_ind])
        else:
            pass    

        #iterate through elements
        if row_dir=='down':
            row_ind+=1
        else:
            row_ind-=1
        col_ind+=1

    return ''.join(plaintext_list)

### Run

In [5]:
plaintext='Au fait, DS Model is an exceptional model to calibrate the base rate bias. In its defense, the poor performance in our task is caused by the lack of diverse opinions. Financial industry does not have a diverse pool of participants. The candidates are generally screened by target university and private school at the recruitment stage. Especially when it comes to research analysts, they don’t have any collective knowledge and they forecast the market based on public polling data. To add more fuel to the fire, research analysts do not get rated by conducting excellent research or telling fascinating stories. Their rating is based upon clients’ perception. Their ability to entertain the clients is somehow more influential than their research capability (one big night with the client is worth a thousand reports). To put it plainly, there are too many analysts who cannot do Newton-Leibniz theorem right and you expect them to tell you what the price would be in one year’s time? They are good at Latin quotes used in a variety of ways in the rhetoric but beautiful anecdotical stories doesn’t mean shit in terms of proofs.'
num_of_rail=4

In [6]:
#as we will use regex later,we have to remove space here
#encrypt
ciphertext=rail_fence_cipher_encrypt(plaintext.replace(' ',''),
                                     num_of_rail=num_of_rail)
print(ciphertext)

AGh&[U,m,6sce@if-deh4E<7i%{2:5o?{CrKc``/C6aAKSwlb&or@?tywH"C.EGEPPdHon<We+=%sjoR.h<(fFwxD;c^,Hm{rwEt7dsZ_4:cdNb*3zl"YEeXduUZoBeH'[k=oTfT_*nM]W3_l+G20$t3QE0As8_MBHvy0J,:edH8vzo|-kM9r>0jboalGGX<h,]:tbiTW/3raak~d%erp5Nfsvsw)*eN+mB+r>f]?@iopE^)tIKy0'r'-S7ysZh3=Ca(=98LeUP}L)mDzQH1a|GAa5p,\aj}l#s0ztiiHAu~sl9QXeeyjjI3nBlEV3s>`:R%dv+{=taQG'fWc6sdsmt$[m)4o+%)8te@w(JJe=pU8Pco[3Mjelkr/JtVv"{no>2TN$iz@KG[i9oT{Na>'cLZdR"_{~uQRvMrhO<Xgl,!Z|ubra7/C"lqy_eZoNk#{>tX?3FzbT^-`ku*X]{Fe';1ZXe]%p5_eJgTxarxc5Ynn6rD/Ji+"I)WgG~<K[e]"u'vii>*3'nMCF#Cs~o1mHnJ(yPUtO|k6bcVzc10nN't#%r;ucLXt:%6K!t1g*v;n0%Le(i%p]qzs#gRIsoY8WR-ilfJZenOg411h-J];~i14<vZa%Z(+[p:f(HLt`;X(Mb=K].ihd6KpitB]4"<e*Aj@]oOh&OSh5m49rd+RRuPtOMiA3pKJJKolj|(6/,M}zOian}#2smP$:95aP7;`Rw=xg&xn/NHO#e,@aE/Lu"%"Vz9YW&$eRx1Rzty#ekTux)^k!t>2qOGouw9t>o-k=+nt.kUuscN>LQwdW*YIQn:&aiL’WNv]k?ZFM$rrTf}+*a3c4UWn%qB=fs-4zV8nk&m5eeT>w-}a}OR-:hD1Z|poIS^N+t%6y$LiC(m+Pe`"VLmc@LMWjrC>K8`e8}*[le[F{#5t5G*`:m^l#3Vo1oP@!u4&RtHDNSEd<l]Q!nrx#LBtqoGg^m.dvHWoBa\A(ryt}1]e|ad"DaoeIOQ

In [7]:
#only capture non-empty characters
#break into blocks
ultimate_ciphertext=break_into_blocks(ciphertext,bandwidth=8)
print(ultimate_ciphertext)

agh&[u,m ,6sce@if -deh4e<7 i%{2:5o? {crkc``/ c6aakswl b&or@?ty wh"c.ege ppdhon<w e+=%sjor .h<(ffwx d;c^,hm{ rwet7dsz _4:cdnb* 3zl"yeex duuzobeh '[k=otft _*nm]w3_ l+g20$t3 qe0as8_m bhvy0j,: edh8vzo| -km9r>0j boalggx< h,]:tbit w/3raak~ d%erp5nf svsw)*en +mb+r>f] ?@iope^) tiky0'r' -s7yszh3 =ca(=98l eup}l)md zqh1a|ga a5p,\aj} l#s0ztii hau~sl9q xeeyjji3 nblev3s> `:r%dv+{ =taqg'fw c6sdsmt$ [m)4o+%) 8te@w(jj e=pu8pco jelkr /jtvv"{n o>2tn$iz @kg[i9ot {na>'clz dr"_{~uq rvmrho<x gl,!z|ub ra7/c"lq y_ezonk# {>tx?3fz bt^-`ku* x]{fe';1 zxe]%p5_ ejgtxarx c5ynn6rd /ji+"i)w gg~<k[e] "u'vii>* 3'nmcf#c s~o1mhnj (yputo|k 6bcvzc10 nn't#%r; uclxt:%6 k!t1g*v; n0%le(i% p]qzs#gr isoy8wr- ilfjzeno g411h-j] ;~i14<vz a%z(+[p: f(hlt`;x (mb=k].i hd6kpitb ]4"<e*aj @]ooh&os h5m49rd+ rruptomi a3pkjjko lj|(6/,m }zoian}# 2smp$:95 ap7;`rw= xg&xn/nh o#e,@ae/ lu"%"vz9 yw&$erx1 rzty#ekt ux)^k!t> 2qogouw9 t>o-k=+n t.kuuscn >lqwdw*y iqn:&ail ’wnv]k?z fm$rrtf} +*a3c4uw n%qb=fs- 4zv8nk&m 5eet>w-} a}or-:hd 1z|pois^ n+t%6y$l ic(m

In [8]:
#remove padding
ind=0
for i in ultimate_ciphertext[::-1]:
    if i!='a':
        break
    ind+=1

In [9]:
#decrypt
rail_fence_cipher_decrypt(ultimate_ciphertext.replace(' ','')[:-ind],
                          num_of_rail=num_of_rail)

'aufait,dsmodelisanexceptionalmodeltocalibratethebaseratebias.initsdefense,thepoorperformanceinourtaskiscausedbythelackofdiverseopinions.financialindustrydoesnothaveadiversepoolofparticipants.thecandidatesaregenerallyscreenedbytargetuniversityandprivateschoolattherecruitmentstage.especiallywhenitcomestoresearchanalysts,theydon’thaveanycollectiveknowledgeandtheyforecastthemarketbasedonpublicpollingdata.toaddmorefueltothefire,researchanalystsdonotgetratedbyconductingexcellentresearchortellingfascinatingstories.theirratingisbaseduponclients’perception.theirabilitytoentertaintheclientsissomehowmoreinfluentialthantheirresearchcapability(onebignightwiththeclientisworthathousandreports).toputitplainly,therearetoomanyanalystswhocannotdonewton-leibniztheoremrightandyouexpectthemtotellyouwhatthepricewouldbeinoneyear’stime?theyaregoodatlatinquotesusedinavarietyofwaysintherhetoricbutbeautifulanecdoticalstoriesdoesn’tmeanshitintermsofproofs.'

### Cryptanalysis

&nbsp;

There is no smart way to crack rail fence cipher without knowing the number of rails. The easiest way is a smart brute force. First, we obtain all the possible combinations of the factors of the length of the ciphertext. Next, we try every possible value and check if zigzag pattern has showed any substantial meaning of a natural language.

In [10]:
#you can check the following link for how factorization is done in recursion
# https://github.com/je-suis-tm/recursion-and-dynamic-programming/blob/master/factorization.py
global factors
factors=[]
def factorization(n):
    
    global factors
    
    #negative and float should be excluded
    assert n>=0 and type(n)!=float,"negative or float is not allowed"
        
    #if n is smaller than 4 
    #prime number it is
    if n>4:
        
        #exclude 1 and itself
        #the largest factor of n can not exceed the half of n
        #because 2 is the smallest factor
        #the range of factors we are gonna try starts from 2 to int(n/2+1)
        #int function to solve the odd number problem
        for i in range(2,int(n**0.5)+1):
            
            #the factorization process
            if n%i==0:
                factors.append(i)
                
                #return is crucial
                #if the number is not a prime number
                #it will stop function from appending non-prime factors
                #the next few lines will be ignored
                return f(int(n/i))
    
    #append the last factor    
    #it could be n itself if n is a prime number
    #in that case there is only one element in the list
    #or it could be the last indivisible factor of n which is also a prime number
    factors.append(int(n))
    
    if len(factors)==1:
        print('%d is a prime number'%(n))
        factors=[]

In [11]:
#number of rails must be the combinations of the following factors
#then we can brute force
factorization(len(ultimate_ciphertext.replace(' ','')))
print(factors)

NameError: name 'f' is not defined

In [ ]:
#find all the combinations of the factors
combinations=[]
for i in range(1,len(factors)):
    combinations+=list(itertools.combinations(factors,i))

#multiplication of the factors
possible_num_of_rails=[]
for i in set(combinations):
    product=1
    for j in i:
        product*=j
    possible_num_of_rails.append(product)

#brute force all possible values
print(possible_num_of_rails)